# Simple Chains trong LangChain

## Khái niệm về Chains

### Chains là gì?
**Chain** trong LangChain là cách để kết nối nhiều components lại với nhau thành một workflow có thể tái sử dụng. Chains cho phép:

- **Kết nối** các components như Prompts, LLMs, Output Parsers
- **Tự động hóa** data flow giữa các bước
- **Tái sử dụng** logic xử lý phức tạp
- **Compose** các chains nhỏ thành chains lớn hơn

### Tại sao cần Chains?
1. **Modularity**: Chia nhỏ complex tasks thành manageable pieces
2. **Reusability**: Sử dụng lại chains cho different inputs
3. **Maintainability**: Dễ debug và modify từng component
4. **Composability**: Kết hợp chains để tạo workflows phức tạp

### Chain Operators trong LangChain
LangChain sử dụng **pipe operator** (`|`) để chain components:
```python
chain = prompt | llm | output_parser
```

Điều này tương đương với:
```python
def chain(input_data):
    prompt_output = prompt.format(**input_data)
    llm_output = llm.invoke(prompt_output)
    final_output = output_parser.parse(llm_output)
    return final_output
```

## Setup và Import

In [ ]:
# Import các thư viện cần thiết
import os
from dotenv import load_dotenv

# LangChain Core
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# LangChain Anthropic
from langchain_anthropic import ChatAnthropic

# Output Parsers
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Utilities
import json
from typing import List, Dict

# Load environment variables
load_dotenv()

print("✓ Đã import các thư viện cần thiết")

In [ ]:
# Khởi tạo ChatAnthropic
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.7,
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
)

print("✓ LLM đã được khởi tạo")

## 1. Chain đơn giản nhất: Prompt + LLM

In [ ]:
# Tạo prompt template đơn giản
simple_prompt = ChatPromptTemplate.from_template(
    "Viết một câu thơ ngắn về {topic}. Hãy sáng tạo và có cảm xúc."
)

# Tạo chain đơn giản: prompt + llm
simple_chain = simple_prompt | llm

print("✓ Simple chain đã được tạo")
print("Chain structure: prompt | llm")

# Test chain
result = simple_chain.invoke({"topic": "mùa xuân"})

print(f"\n=== Test Simple Chain ===")
print(f"Input: topic='mùa xuân'")
print(f"Output type: {type(result)}")
print(f"Output: {result.content}")

In [ ]:
# Test với nhiều topics khác nhau
topics = ["biển cả", "núi rừng", "thành phố", "tình yêu"]

print("=== Testing với nhiều topics ===")
for topic in topics:
    result = simple_chain.invoke({"topic": topic})
    print(f"\n🎨 Topic: {topic}")
    print(f"📝 Thơ: {result.content}")

## 2. Chain với Output Parser: Prompt + LLM + Parser

In [ ]:
# Chain với String Output Parser
str_parser = StrOutputParser()

# Chain hoàn chỉnh: prompt + llm + parser
string_chain = simple_prompt | llm | str_parser

print("✓ String chain đã được tạo")
print("Chain structure: prompt | llm | str_parser")

# Test và so sánh
test_topic = "mùa thu"

# Without parser
without_parser = simple_chain.invoke({"topic": test_topic})
print(f"\n=== So sánh Output ===")
print(f"Without parser - Type: {type(without_parser)}")
print(f"Without parser - Content: {without_parser.content}")

# With parser
with_parser = string_chain.invoke({"topic": test_topic})
print(f"\nWith parser - Type: {type(with_parser)}")
print(f"With parser - Content: {with_parser}")

In [ ]:
# Chain với List Output Parser
list_prompt = ChatPromptTemplate.from_template(
    """Liệt kê 5 điều thú vị về {subject}.
    
    {format_instructions}"""
)

list_parser = CommaSeparatedListOutputParser()

# Chain với list parser
list_chain = list_prompt | llm | list_parser

print("✓ List chain đã được tạo")
print(f"Format instructions: {list_parser.get_format_instructions()}")

# Test list chain
subjects = ["vũ trụ", "đại dương", "trí tuệ nhân tạo"]

print(f"\n=== Testing List Chain ===")
for subject in subjects:
    result = list_chain.invoke({
        "subject": subject,
        "format_instructions": list_parser.get_format_instructions()
    })
    
    print(f"\n📚 Subject: {subject}")
    print(f"📋 Type: {type(result)}")
    print(f"📝 List items:")
    for i, item in enumerate(result, 1):
        print(f"   {i}. {item}")

## 3. Chain với Structured Output

In [ ]:
# Định nghĩa Pydantic model cho structured output
class ProductReview(BaseModel):
    product_name: str = Field(description="Tên sản phẩm")
    rating: int = Field(description="Đánh giá từ 1-5 sao", ge=1, le=5)
    pros: List[str] = Field(description="Các ưu điểm")
    cons: List[str] = Field(description="Các nhược điểm")
    summary: str = Field(description="Tóm tắt đánh giá")
    would_recommend: bool = Field(description="Có recommend hay không")

# Tạo parser từ Pydantic model
pydantic_parser = PydanticOutputParser(pydantic_object=ProductReview)

# Prompt cho product review
review_prompt = ChatPromptTemplate.from_template(
    """Phân tích đánh giá sản phẩm sau và trích xuất thông tin có cấu trúc:
    
    Đánh giá: {review_text}
    
    {format_instructions}"""
)

# Tạo structured chain
structured_chain = review_prompt | llm | pydantic_parser

print("✓ Structured chain đã được tạo")
print("Chain structure: prompt | llm | pydantic_parser")

In [ ]:
# Test structured chain
sample_review = """
iPhone 15 Pro Max thực sự ấn tượng! Màn hình 6.7 inch siêu sắc nét, 
camera 48MP chụp ảnh cực đẹp, đặc biệt là chế độ chụp đêm. 
Pin sử dụng được cả ngày mà không cần sạc. 
Tuy nhiên giá hơi cao, khoảng 35 triệu, và hơi nặng so với các dòng trước. 
Nhìn chung mình rất hài lòng và sẽ recommend cho bạn bè.
"""

result = structured_chain.invoke({
    "review_text": sample_review,
    "format_instructions": pydantic_parser.get_format_instructions()
})

print("=== Structured Output ===")
print(f"Type: {type(result)}")
print(f"\n📱 Product: {result.product_name}")
print(f"⭐ Rating: {result.rating}/5")
print(f"✅ Would recommend: {result.would_recommend}")
print(f"\n🟢 Pros:")
for pro in result.pros:
    print(f"   • {pro}")
print(f"\n🔴 Cons:")
for con in result.cons:
    print(f"   • {con}")
print(f"\n📝 Summary: {result.summary}")

## 4. Chain với Conditional Logic

In [ ]:
# Tạo conditional logic với RunnableLambda
def determine_response_style(input_dict):
    """Xác định style phản hồi dựa trên audience"""
    audience = input_dict.get("audience", "general")
    topic = input_dict.get("topic", "")
    
    if audience == "kids":
        style = "đơn giản, vui nhộn, dễ hiểu"
    elif audience == "experts":
        style = "chuyên sâu, kỹ thuật, chi tiết"
    elif audience == "business":
        style = "chuyên nghiệp, súc tích, tập trung vào giá trị"
    else:
        style = "cân bằng, dễ hiểu, thông tin đầy đủ"
    
    return {
        "topic": topic,
        "audience": audience,
        "style": style
    }

# Prompt động dựa trên style
adaptive_prompt = ChatPromptTemplate.from_template(
    """Giải thích về {topic} cho đối tượng {audience}.
    
    Style yêu cầu: {style}
    
    Hãy điều chỉnh ngôn ngữ và độ phức tạp cho phù hợp."""
)

# Tạo conditional chain
conditional_chain = (
    RunnableLambda(determine_response_style) 
    | adaptive_prompt 
    | llm 
    | StrOutputParser()
)

print("✓ Conditional chain đã được tạo")
print("Chain structure: style_determiner | prompt | llm | parser")

In [ ]:
# Test conditional chain với different audiences
test_cases = [
    {"topic": "trí tuệ nhân tạo", "audience": "kids"},
    {"topic": "trí tuệ nhân tạo", "audience": "experts"},
    {"topic": "trí tuệ nhân tạo", "audience": "business"},
    {"topic": "trí tuệ nhân tạo", "audience": "general"}
]

print("=== Testing Conditional Chain ===")
for test_case in test_cases:
    result = conditional_chain.invoke(test_case)
    
    print(f"\n🎯 Audience: {test_case['audience']}")
    print(f"📖 Topic: {test_case['topic']}")
    print(f"💬 Response: {result[:200]}...")
    print("-" * 60)

## 5. Chain với Data Processing

In [ ]:
# Data processing functions
def extract_keywords(input_dict):
    """Extract keywords từ text"""
    text = input_dict["text"]
    # Simplified keyword extraction
    words = text.lower().split()
    # Filter out common words (simplified)
    stop_words = {"và", "của", "trong", "với", "từ", "để", "có", "là", "một", "các", "này", "đó"}
    keywords = [word for word in words if len(word) > 3 and word not in stop_words]
    # Take top 5 unique keywords
    unique_keywords = list(dict.fromkeys(keywords))[:5]
    
    return {
        "original_text": text,
        "keywords": unique_keywords
    }

def format_summary_prompt(input_dict):
    """Format prompt với keywords"""
    return {
        "text": input_dict["original_text"],
        "keywords": ", ".join(input_dict["keywords"])
    }

# Prompt cho summarization
summary_prompt = ChatPromptTemplate.from_template(
    """Tóm tắt văn bản sau thành 2-3 câu, tập trung vào các từ khóa chính: {keywords}
    
    Văn bản: {text}
    
    Tóm tắt:"""
)

# Tạo processing chain
processing_chain = (
    RunnableLambda(extract_keywords)
    | RunnableLambda(format_summary_prompt)
    | summary_prompt
    | llm
    | StrOutputParser()
)

print("✓ Processing chain đã được tạo")
print("Chain structure: keyword_extract | format | prompt | llm | parser")

In [ ]:
# Test processing chain
sample_text = """
LangChain là một framework mạnh mẽ được thiết kế để phát triển các ứng dụng 
sử dụng Large Language Models (LLMs). Framework này cung cấp nhiều công cụ 
và abstractions giúp developers dễ dàng tích hợp LLMs vào các ứng dụng thực tế. 
LangChain hỗ trợ nhiều LLM providers khác nhau như OpenAI, Anthropic, Cohere, 
và Hugging Face. Các thành phần chính của LangChain bao gồm Chains, Agents, 
Memory, và Callbacks. Chains cho phép kết nối nhiều components lại với nhau 
để tạo ra workflows phức tạp. Agents có khả năng reasoning và tự quyết định 
actions cần thực hiện. Memory giúp lưu trữ conversation history để maintain 
context giữa các lần tương tác.
"""

# Test keyword extraction step
keywords_result = extract_keywords({"text": sample_text})
print("=== Keyword Extraction ===")
print(f"Keywords: {keywords_result['keywords']}")

# Test full chain
summary_result = processing_chain.invoke({"text": sample_text})
print(f"\n=== Summary Result ===")
print(f"Original length: {len(sample_text)} characters")
print(f"Summary: {summary_result}")
print(f"Summary length: {len(summary_result)} characters")
print(f"Compression ratio: {len(summary_result)/len(sample_text):.2%}")

## 6. Chain với Multiple Outputs

In [ ]:
# Tạo chain trả về multiple outputs
from langchain_core.runnables import RunnableParallel

# Different prompts cho different tasks
sentiment_prompt = ChatPromptTemplate.from_template(
    "Phân tích cảm xúc của text sau (Tích cực/Tiêu cực/Trung lập): {text}"
)

category_prompt = ChatPromptTemplate.from_template(
    "Phân loại chủ đề của text sau (Công nghệ/Kinh doanh/Giải trí/Khác): {text}"
)

language_prompt = ChatPromptTemplate.from_template(
    "Đánh giá độ phức tạp ngôn ngữ của text sau (Đơn giản/Trung bình/Phức tạp): {text}"
)

# Tạo parallel chains
parallel_chain = RunnableParallel(
    sentiment=sentiment_prompt | llm | StrOutputParser(),
    category=category_prompt | llm | StrOutputParser(),
    complexity=language_prompt | llm | StrOutputParser(),
    original_text=RunnablePassthrough()
)

print("✓ Parallel chain đã được tạo")
print("Chain structure: parallel(sentiment, category, complexity, original)")

In [ ]:
# Test parallel chain
test_texts = [
    "Tôi rất hào hứng với công nghệ AI mới này! Nó sẽ thay đổi cách chúng ta làm việc.",
    "Thị trường chứng khoán hôm nay giảm mạnh, nhiều nhà đầu tư lo lắng về triển vọng kinh tế.",
    "Bộ phim hôm qua khá hay, diễn viên diễn xuất tự nhiên và kịch bản hấp dẫn."
]

print("=== Testing Parallel Chain ===")
for i, text in enumerate(test_texts, 1):
    result = parallel_chain.invoke({"text": text})
    
    print(f"\n📄 Text {i}: {text[:80]}...")
    print(f"😊 Sentiment: {result['sentiment']}")
    print(f"📂 Category: {result['category']}")
    print(f"🎯 Complexity: {result['complexity']}")
    print("-" * 60)

## 7. Chain Composition - Kết hợp chains

In [ ]:
# Tạo analysis chain từ parallel chain
def combine_analysis(parallel_result):
    """Combine results từ parallel analysis"""
    return {
        "text": parallel_result["original_text"]["text"],
        "sentiment": parallel_result["sentiment"],
        "category": parallel_result["category"],
        "complexity": parallel_result["complexity"]
    }

# Final summary prompt
final_summary_prompt = ChatPromptTemplate.from_template(
    """Dựa trên phân tích sau, tạo một báo cáo tóm tắt về văn bản:
    
    Text: {text}
    Cảm xúc: {sentiment}
    Chủ đề: {category}
    Độ phức tạp: {complexity}
    
    Báo cáo tóm tắt (2-3 câu):"""
)

# Composed chain: parallel analysis → combine → final summary
composed_chain = (
    parallel_chain
    | RunnableLambda(combine_analysis)
    | final_summary_prompt
    | llm
    | StrOutputParser()
)

print("✓ Composed chain đã được tạo")
print("Chain: parallel_analysis | combine | summary_prompt | llm | parser")

In [ ]:
# Test composed chain
complex_text = """
Việc ứng dụng trí tuệ nhân tạo trong chăm sóc sức khỏe đang mở ra những 
cơ hội to lớn nhưng cũng đặt ra nhiều thách thức về đạo đức và quyền riêng tư. 
Các thuật toán machine learning có thể giúp chẩn đoán bệnh chính xác hơn, 
nhưng việc xử lý dữ liệu y tế nhạy cảm cần được thực hiện cẩn trọng. 
Sự hợp tác giữa các chuyên gia y tế, kỹ sư phần mềm và các nhà hoạch định 
chính sách là cần thiết để đảm bảo AI được sử dụng một cách có trách nhiệm 
và mang lại lợi ích tối đa cho bệnh nhân.
"""

print("=== Testing Composed Chain ===")
print(f"Input text: {complex_text[:100]}...")
print(f"\n📊 Running parallel analysis...")

# Test parallel analysis first
parallel_result = parallel_chain.invoke({"text": complex_text})
print(f"Sentiment: {parallel_result['sentiment']}")
print(f"Category: {parallel_result['category']}")
print(f"Complexity: {parallel_result['complexity']}")

# Test full composed chain
print(f"\n📝 Generating final summary...")
final_result = composed_chain.invoke({"text": complex_text})
print(f"\n📋 Final Summary:\n{final_result}")

## 8. Error Handling trong Chains

In [ ]:
# Chain với error handling
def safe_text_processor(input_dict):
    """Process text với error handling"""
    try:
        text = input_dict.get("text", "")
        if not text or len(text.strip()) == 0:
            return {"error": "Empty text provided", "processed_text": ""}
        
        if len(text) > 5000:
            return {
                "warning": "Text too long, truncating",
                "processed_text": text[:5000] + "..."
            }
        
        return {"processed_text": text, "status": "success"}
        
    except Exception as e:
        return {"error": str(e), "processed_text": ""}

def handle_processing_result(result):
    """Handle result từ text processor"""
    if "error" in result:
        return {"text": f"Error: {result['error']}"}
    
    if "warning" in result:
        print(f"⚠️ Warning: {result['warning']}")
    
    return {"text": result["processed_text"]}

# Safe chain với error handling
safe_prompt = ChatPromptTemplate.from_template(
    "Phân tích và tóm tắt văn bản sau: {text}"
)

safe_chain = (
    RunnableLambda(safe_text_processor)
    | RunnableLambda(handle_processing_result)
    | safe_prompt
    | llm
    | StrOutputParser()
)

print("✓ Safe chain với error handling đã được tạo")

In [ ]:
# Test error handling
test_cases_error = [
    {"text": ""},  # Empty text
    {"text": "   "},  # Whitespace only
    {"text": "Normal text để test."},  # Normal case
    {"text": "A" * 6000},  # Very long text
]

print("=== Testing Error Handling ===")
for i, test_case in enumerate(test_cases_error, 1):
    try:
        result = safe_chain.invoke(test_case)
        print(f"\n✅ Test {i}: Success")
        if len(str(result)) > 200:
            print(f"Result: {str(result)[:200]}...")
        else:
            print(f"Result: {result}")
    except Exception as e:
        print(f"\n❌ Test {i}: Error - {str(e)}")

## 9. Chain Performance và Monitoring

In [ ]:
import time
from functools import wraps

# Performance monitoring wrapper
def monitor_performance(chain_name):
    """Decorator để monitor chain performance"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            try:
                result = func(*args, **kwargs)
                end_time = time.time()
                duration = end_time - start_time
                print(f"⏱️ {chain_name}: {duration:.2f}s")
                return result
            except Exception as e:
                end_time = time.time()
                duration = end_time - start_time
                print(f"❌ {chain_name}: Failed after {duration:.2f}s - {str(e)}")
                raise
        return wrapper
    return decorator

# Benchmark different chains
@monitor_performance("Simple Chain")
def run_simple_chain(input_data):
    return string_chain.invoke(input_data)

@monitor_performance("Structured Chain")
def run_structured_chain(input_data):
    return structured_chain.invoke(input_data)

@monitor_performance("Parallel Chain")
def run_parallel_chain(input_data):
    return parallel_chain.invoke(input_data)

# Test performance
test_input = {"topic": "blockchain technology"}
test_review = {
    "review_text": "Sản phẩm tốt, giao hàng nhanh, giá hợp lý. Recommend!",
    "format_instructions": pydantic_parser.get_format_instructions()
}
test_text = {"text": "AI đang thay đổi thế giới một cách tích cực."}

print("=== Performance Benchmark ===")
print("Testing chain performance...")

# Run benchmarks
simple_result = run_simple_chain(test_input)
structured_result = run_structured_chain(test_review)
parallel_result = run_parallel_chain(test_text)

print("\n✅ All benchmarks completed")

## 10. Best Practices cho Simple Chains

In [ ]:
# Best practices demonstration
def demonstrate_best_practices():
    print("=== SIMPLE CHAINS BEST PRACTICES ===")
    
    practices = [
        {
            "title": "1. START SIMPLE",
            "description": "Begin với basic prompt | llm chain",
            "example": "prompt | llm | parser"
        },
        {
            "title": "2. ADD PARSERS",
            "description": "Always use output parsers for consistent results",
            "example": "StrOutputParser, JsonOutputParser, PydanticOutputParser"
        },
        {
            "title": "3. ERROR HANDLING",
            "description": "Implement graceful error handling",
            "example": "RunnableLambda với try-catch blocks"
        },
        {
            "title": "4. COMPOSABILITY",
            "description": "Design chains to be composable",
            "example": "Small chains → Combined chains"
        },
        {
            "title": "5. MONITORING",
            "description": "Monitor performance và errors",
            "example": "Timing, logging, error tracking"
        }
    ]
    
    for practice in practices:
        print(f"\n{practice['title']}")
        print(f"   {practice['description']}")
        print(f"   Example: {practice['example']}")

demonstrate_best_practices()

In [ ]:
# Common pitfalls và solutions
def common_pitfalls():
    print("\n=== COMMON PITFALLS & SOLUTIONS ===")
    
    pitfalls = [
        {
            "problem": "❌ No Output Parsing",
            "issue": "Raw LLM output không consistent",
            "solution": "✅ Always use appropriate parsers"
        },
        {
            "problem": "❌ Complex Single Chain",
            "issue": "Monolithic chain khó debug",
            "solution": "✅ Break into smaller, testable chains"
        },
        {
            "problem": "❌ No Error Handling",
            "issue": "Chain fails với invalid input",
            "solution": "✅ Implement input validation và error handling"
        },
        {
            "problem": "❌ Hardcoded Values",
            "issue": "Chain không flexible",
            "solution": "✅ Use parameters và conditional logic"
        },
        {
            "problem": "❌ No Testing",
            "issue": "Bugs in production",
            "solution": "✅ Test each component và full chain"
        }
    ]
    
    for pitfall in pitfalls:
        print(f"\n{pitfall['problem']}")
        print(f"   Issue: {pitfall['issue']}")
        print(f"   {pitfall['solution']}")

common_pitfalls()

print("\n✅ Simple Chains tutorial completed!")

## Tổng kết

### **Simple Chains: Key Concepts**

#### **Chain Basics**
- **Purpose**: Kết nối components thành reusable workflows
- **Operator**: Pipe (`|`) để chain components
- **Flow**: Data tự động flow từ component này sang component khác

#### **Chain Patterns học được**
1. **Basic Chain**: `prompt | llm`
2. **With Parser**: `prompt | llm | parser`
3. **Conditional**: `logic | prompt | llm | parser`
4. **Parallel**: `RunnableParallel` cho multiple outputs
5. **Composed**: Kết hợp multiple chains

#### **Components đã sử dụng**
- **ChatPromptTemplate**: Structured prompts
- **ChatAnthropic**: LLM for generation
- **Output Parsers**: String, List, Pydantic
- **RunnableLambda**: Custom logic
- **RunnableParallel**: Parallel execution

### **Best Practices**
1. **Start Simple**: Begin với basic chain, add complexity gradually
2. **Always Parse**: Use output parsers cho consistent results
3. **Error Handling**: Implement graceful error handling
4. **Composability**: Design for reusability và composition
5. **Testing**: Test individual components và full chains
6. **Monitoring**: Track performance và errors

### **Next Steps**
- **Sequential Chains**: Multi-step workflows
- **Retrieval Chains**: Combine với vectorstores
- **Agent Chains**: Decision-making workflows
- **Production**: Deployment, scaling, monitoring

Simple Chains là foundation cho complex LangChain applications!